# Cooling Effect Module Draft 

In [1]:
# Install the GUS framework.

import sys
!{sys.executable} -m pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple gus==0.1.7

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple


In [26]:
import numpy as np
import pandas as pd

In [18]:
DBH=15
TH=5
TCD=4
TCDth=3

In [79]:
def compute_leaf_area_index(
    dbh,
    tree_height,
    crown_depth,
    crown_diameter,
    crown_missing=0,
    shade_factor=0.85,
):
    """The function given allometrics of a tree computes its leaf, bark and plant area indices.

    Args:
        dbh: (:obj:`float`): the diameter in cm of the trunk usually measured at 1.3m from the ground.
        tree_height: (:obj:`float`): The tree height in meters.
        crown_debth: (:obj:`float`): The vertical length of tree crown in meters.
        crown_diameter: (:obj:`float`): The horizontal length (diameter) of tree crown in meters.
        crown_missing: (:obj:`float`): The percentage loss of the crown.
        shade_factor: (:obj:`float`): The percentage of sky covered by foliage and branches.

        Shade factor, is the percentage of sky covered by foliage and branches within the perimeter of individual tree crowns,
        can vary by species from about 60% to 95% when trees are in-leaf (McPherson, 1984). The value below is set according to 
        Glasgow mean and needs to be updated per city or per tree.

    Returns:
        (:obj:`float`): It eturns the LAI.
    Note:
        The beta multipliers and the main equation is based on Nowak (1996).

    TODO:
        Parametrize beta multipliers.
    """
    loss = crown_missing
    th = tree_height
    cw = crown_diameter
    cd = crown_depth
    sf = shade_factor
    beta_0 = -4.3309
    beta_1 = 0.2942
    beta_2 = 0.7312
    beta_3 = 5.7217
    beta_4 = 0.0148

    def compute_under_canopy_area(crown_diameter):
        return pow((crown_diameter / 2), 2) * np.pi

    # Outer surface area estimate below is based on Gacka-Grzesikiewicz (1980).
    under_canopy = compute_under_canopy_area(cw)
    crown_surface = np.pi * cw * (cd + cw) / 2
    leaf_area = (1 - loss) * np.exp(
        beta_0 + beta_1 * th + beta_2 * cw + beta_3 * sf - beta_4 * crown_surface
    )
    leaf_area_index = leaf_area / under_canopy
    return leaf_area_index

In [80]:
compute_leaf_area_index(dbh=DBH, tree_height=TH, crown_diameter=TCD, crown_depth=TCDth)

5.733824472020663

In [81]:
def compute_gross_canopy_index(
    tree_height,
    crown_depth,
    crown_diameter
):
    """The function given allometrics of a tree computes an index to account shading effect immediately underneath its canopy.

    Args:
        tree_height: (:obj:`float`): The tree height in meters.
        crown_height: (:obj:`float`): The vertical length of tree crown in meters.
        crown_width: (:obj:`float`): The horizontal length (diameter) of tree crown in meters.

    Returns:
        (:obj:`float`): returns the GCI index
    Note:
        The index is based on (Zhang et.al., 2020). It accounts for the overall immediate shading effect of a tree.
        It accounts both crown diameter (tw) and the crown base height (th - ch), the distance from the ground to the start of
        the canopy.
    """
    th = tree_height
    cw = crown_diameter
    cd = crown_depth
    crown_base_height = th  - cd
    if crown_base_height < 0.1:
        crown_base_height = 0.1 
    GCI = cw / crown_base_height
    return GCI

In [82]:
compute_gross_canopy_index(tree_height = TH, crown_depth = TCDth, crown_diameter = TCD)

2.0

In [83]:
def compute_crown_volume_index(
    leaf_area_index,
    crown_canopy_index
):
    """The function given computes a composite index taking into account both leaf density and canopy shape.

    Args:
        leaf_area_indext: (:obj:`float`): The index on the leaf density per the area under the canopy.
        crown_canopy_index: (:obj:`float`): The index that accounts for the geomtric shape of the canopy relevant to shading.
        
    Returns:
        (:obj:`float`): returns the CVI index
    Note:
        The index is adopted from (Zhang et.al., 2020).
    """
    return leaf_area_index * crown_canopy_index

In [84]:
LAI = compute_leaf_area_index(DBH,TH,TCD,TCDth)
GCI = compute_gross_canopy_index(TH,TCDth,TCD)
compute_crown_volume_index(LAI,GCI)

11.547033684456904

In [85]:
df = pd.read_csv("trees_yearly.csv")

In [86]:
df.tail()

,Step,AgentID,species,dbh,height,crownH,crownW,canopy_overlap,cle,condition,dieback,biomass,seq,carbon,deroot,detrunk,mulched,burnt,coordinates
6906,50,168,decidu,11.318872,9.308317,4.837098,4.188620,0.307545,0.836917,excellent,0.005313,55.669067,5.937846,27.834534,0.0,0.0,0.007395,0.0,"(5, 8)"
6907,50,169,decidu,12.240165,9.712507,5.082395,4.401332,0.163847,0.917741,excellent,0.002980,66.387401,7.189507,33.193700,0.0,0.0,0.004946,0.0,"(2, 10)"
6908,50,170,conifer,10.196881,6.444752,4.071138,2.555131,0.069372,0.959620,excellent,0.000094,36.809711,2.961742,18.404856,0.0,0.0,0.000087,0.0,"(14, 5)"
6909,50,171,conifer,10.228828,6.462722,4.079543,2.561090,0.200136,0.883662,excellent,0.005502,37.092435,2.737088,18.546217,0.0,0.0,0.005102,0.0,"(1, 5)"
6910,50,172,decidu,9.197756,8.316054,4.257468,3.688400,0.000000,0.560000,excellent,0.008239,34.899665,0.000000,17.449832,0.0,0.0,0.000000,0.0,"(6, 3)"


In [87]:
df.set_index("AgentID", inplace=True)
df.rename(columns={'Step': 'year'}, inplace=True)

In [88]:
df.tail()

,year,species,dbh,height,crownH,crownW,canopy_overlap,cle,condition,dieback,biomass,seq,carbon,deroot,detrunk,mulched,burnt,coordinates
AgentID,,,,,,,,,,,,,,,,,,
168,50,decidu,11.318872,9.308317,4.837098,4.188620,0.307545,0.836917,excellent,0.005313,55.669067,5.937846,27.834534,0.0,0.0,0.007395,0.0,"(5, 8)"
169,50,decidu,12.240165,9.712507,5.082395,4.401332,0.163847,0.917741,excellent,0.002980,66.387401,7.189507,33.193700,0.0,0.0,0.004946,0.0,"(2, 10)"
170,50,conifer,10.196881,6.444752,4.071138,2.555131,0.069372,0.959620,excellent,0.000094,36.809711,2.961742,18.404856,0.0,0.0,0.000087,0.0,"(14, 5)"
171,50,conifer,10.228828,6.462722,4.079543,2.561090,0.200136,0.883662,excellent,0.005502,37.092435,2.737088,18.546217,0.0,0.0,0.005102,0.0,"(1, 5)"
172,50,decidu,9.197756,8.316054,4.257468,3.688400,0.000000,0.560000,excellent,0.008239,34.899665,0.000000,17.449832,0.0,0.0,0.000000,0.0,"(6, 3)"


In [97]:
t_42 = df.xs(42)
t_31 = df.xs(31)

In [98]:
t_42 = t_42[['year','species','dbh','height','crownH', 'crownW']]
t_31 = t_31[['year','species','dbh','height','crownH', 'crownW']]

In [99]:
t_31.head()

,year,species,dbh,height,crownH,crownW
AgentID,,,,,,
31,1,decidu,10.160808,8.778266,4.523209,3.917328
31,2,decidu,11.624329,9.443938,4.918861,4.259451
31,3,decidu,13.061136,10.061116,5.297682,4.588556
31,4,decidu,14.468776,10.636331,5.659583,4.904473
31,5,decidu,15.966213,11.220819,6.034541,5.233474


In [100]:
t_31['LAI'] = t_31.apply(lambda row: compute_leaf_area_index(row['dbh'], row['height'],row['crownH'], row['crownW']), axis=1)
t_31.head()

,year,species,dbh,height,crownH,crownW,LAI
AgentID,,,,,,,
31,1,decidu,10.160808,8.778266,4.523209,3.917328,15.203610
31,2,decidu,11.624329,9.443938,4.918861,4.259451,17.459500
31,3,decidu,13.061136,10.061116,5.297682,4.588556,19.836204
31,4,decidu,14.468776,10.636331,5.659583,4.904473,22.302067
31,5,decidu,15.966213,11.220819,6.034541,5.233474,25.050609


In [101]:
t_31['GPI'] = t_31.apply(lambda row: compute_gross_canopy_index(row['height'],row['crownH'], row['crownW']), axis=1)
t_31.head()

,year,species,dbh,height,crownH,crownW,LAI,GPI
AgentID,,,,,,,,
31,1,decidu,10.160808,8.778266,4.523209,3.917328,15.203610,0.920629
31,2,decidu,11.624329,9.443938,4.918861,4.259451,17.459500,0.941299
31,3,decidu,13.061136,10.061116,5.297682,4.588556,19.836204,0.963288
31,4,decidu,14.468776,10.636331,5.659583,4.904473,22.302067,0.985478
31,5,decidu,15.966213,11.220819,6.034541,5.233474,25.050609,1.009100


In [102]:
t_31['CVI'] = t_31.apply(lambda row: compute_crown_volume_index(row['LAI'],row['GPI']), axis=1)
t_31.head()

,year,species,dbh,height,crownH,crownW,LAI,GPI,CVI
AgentID,,,,,,,,,
31,1,decidu,10.160808,8.778266,4.523209,3.917328,15.203610,0.920629,13.996883
31,2,decidu,11.624329,9.443938,4.918861,4.259451,17.459500,0.941299,16.434609
31,3,decidu,13.061136,10.061116,5.297682,4.588556,19.836204,0.963288,19.107969
31,4,decidu,14.468776,10.636331,5.659583,4.904473,22.302067,0.985478,21.978188
31,5,decidu,15.966213,11.220819,6.034541,5.233474,25.050609,1.009100,25.278572
